In [1]:
import pandas as pd
import numpy as np
from ZMiner import *
from ZMinerD import *
from utils import *
import pickle

In [5]:
data_loadtest = pd.read_excel("Synthetic_turbo_failure.xlsx", header=[0, 1], index_col=0)
data_copied = data_loadtest[['no','time','snap','date','status']]
data_copied.columns = ['no','time','snap','date','status']
a = data_loadtest.drop(['no','time','snap','date','status'], axis=1)
a[['no','time','snap','date','status']] = data_copied

# FOR MEMORY SPACE, REMOVE UNUSED VARIABLES
del data_loadtest
del data_copied

/home/dsv/zele5930/.local/lib/python3.6/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [72]:
def test(x):
    #print(int(x['status']))
    if int(x['status']) == 1:
        x['no'] = int(x['no']) + 100
    return x

In [74]:
data_fixed = a.apply(test, axis=1)

In [76]:
z = ZHist(a, ['no','time','snap','date','status'])

/home/dsv/zele5930/.local/lib/python3.6/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [77]:
z.fit()

getWeightedAverage: 0.8265478519606404
interval creation started
current: A
current: B
current: C
current: D
current: E
current: F
current: G
current: H
current: I
current: J
interval creation is done
interval separation started
interval separation is done


In [78]:
database = Database(z.repair_intervals_removed)
constraints = makeConstraints([0.1, 0, 1000], z.repair_intervals_removed)
algorithm = ZMiner(database, constraints, forgettable=True)
algorithm.constraints["timeoutseconds"] = 2000000000
count, freq, timedelta, timeout, FL_repair = algorithm.ZMiner()

########## Z-MINER ##########
1-1. MINIMUM SUPPORT: 10.0
1-2. EPSILON CONSTRAINT: 0.0
1-3. GAP CONSTRAINT: 1000.0
1-4. TIMEOUT: 2000000000
1-5. LEVEL: inf
2. NUMBER OF E-SEQUENCES: 100
3. TOTAL COMPARISON COUNTS: 1265266093
4. TOTAL FREQUENT ARRANGEMENTS: 2200404
5. TOTAL TIME CONSUMED: 13103.802275931


In [79]:
database2 = Database(z.normal_intervals_removed)
constraints2 = makeConstraints([0, 0, 1000], z.normal_intervals_removed)
algorithm2 = ZMinerD(database2, constraints2, FL_repair, forgettable=True)
algorithm2.constraints["timeoutseconds"] = 200000000
coun2t, freq2, tdelta2, timeout2, FL_normal = algorithm2.ZMiner()

########## Z-MINER ##########
1-1. MINIMUM SUPPORT: 0.0
1-2. EPSILON CONSTRAINT: 0.0
1-3. GAP CONSTRAINT: 1000.0
1-4. TIMEOUT: 200000000
2. NUMBER OF E-SEQUENCES: 900
3. TOTAL COMPARISON COUNTS: 139027750
4. TOTAL FREQUENT ARRANGEMENTS: 280255
5. TOTAL TIME CONSUMED: 1842.009999425998


Change some format for csv save

In [80]:
for j in FL_normal[2]:
    for k in FL_normal[2][j]:
        FL_normal[2][j][k] = len(FL_normal[2][j][k])
        
for j in FL_repair[2]:
    for k in FL_repair[2][j]:
        FL_repair[2][j][k] = len(FL_repair[2][j][k])

In [81]:
def exportDisprop(dataname, FL1, FL2, n1, n2, constraints):
    filename = (
        "Disprop"
        + "_"
        + dataname
        + "_"
        + str(constraints["minSupPercent"])
        + "_"
        + str(constraints["epsilon"])
        + "_"
        + str(constraints["gap"])
        + "_"
        + str(constraints["timeoutseconds"])
        + ".csv"
    )
    F = []
    for k in FL1:
        for E in FL1[k]:
            for R in FL1[k][E]:
                if k not in FL2 or E not in FL2[k] or R not in FL2[k][E]:
                    F.append({"events": E, "relations": R, "freq1": FL1[k][E][R], "freq2": 0,
                    "relSup1": (FL1[k][E][R]+1)/(n1+1), "relSup2": 0,
                    "disprop": (((FL1[k][E][R])+1)/(n1+1))/(1/(n2+1))})
                else:
                    F.append({"events": E, "relations": R, "freq1": FL1[k][E][R], "freq2": FL2[k][E][R],
                    "relSup1": (FL1[k][E][R]+1)/(n1+1), "relSup2": (FL2[k][E][R]+1)/(n2+1),
                    "disprop": ((FL1[k][E][R]+1)/(n1+1))/((FL2[k][E][R]+1)/(n2+1))})

    with open(filename, "w") as output_file:
        dict_writer = csv.DictWriter(output_file, ["events", "relations", "freq1", "freq2", "relSup1", "relSup2", "disprop"])
        dict_writer.writeheader()
        dict_writer.writerows(F)
    return filename

In [82]:
# exportF("result1_wholecar", FL_repair, constraints)
exportDisprop("resultWithTrendWithMatrix_syn_SCANIA_v2", FL_repair, FL_normal, 100, 1000, constraints2)

'Disprop_resultWithTrendWithMatrix_syn_turbo_failure_v2_0.0_0.0_1000.0_200000000.csv'